In [1]:
import os
# Find the latest version of spark 3.2 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease [18.1 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease [24.3 kB]
Hit:7 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal InRelease
Get:9 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:10 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main Sources [2,386 kB]
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:13 https://developer.download.nvidia.c

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Amazon_Review").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...|              PC|          5|    

In [4]:
#Vine table
vine_table = df.select(['review_id','star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])

In [5]:
#Filter by voted > 20
total_votes_df = vine_table.filter('total_votes>20')
total_votes_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|
|R35OW0SJPJWAHO|          1|           12|        162|   N|                Y|
| RAP2ETU511BOI|          1|            5|         99|   N|                Y|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [6]:
# Filter on helpful votes > 50%
votes_df = total_votes_df.filter((total_votes_df['helpful_votes'] / total_votes_df['total_votes']) > .5)
votes_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [7]:
#Filter vine == Y
vine_yes_df = votes_df.filter(df['vine'] == 'Y')
vine_yes_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3J2DPYUH6453D|          5|          136|        153|   Y|                N|
|R25QTFLERWK6SJ|          3|          852|        956|   Y|                N|
|R1BG71K9B61LI1|          4|           39|         39|   Y|                N|
| R8WGX426U56E0|          5|           19|         21|   Y|                N|
| R614TCIDD36LE|          4|          131|        147|   Y|                N|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [8]:
#Filter all vine = N
vine_no_df = votes_df.filter(df['vine'] == 'N')
vine_no_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [29]:
#Group all review_id by Vine
total_review = df['vine','review_id'].groupby('vine').count()
total_review.show(5)

+----+-------+
|vine|  count|
+----+-------+
|null|      3|
|   Y|  36230|
|   N|6872321|
+----+-------+



In [10]:
#Filter all 5 star ratings
five_star_rating = df.filter(df['star_rating'] == '5')
five_star_rating.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...|              PC|          5|    

In [11]:
#Group by Paid/Unpaid five star reviews
five_star_rating = five_star_rating.groupby('vine').count()
five_star_rating.show()

+----+-------+
|vine|  count|
+----+-------+
|   Y|  15706|
|   N|4091159|
+----+-------+



In [61]:
# Percentage of paid (vine = Y) 5 star ratings over all ratings
total_fivestar_paid_reviews = vine_table.filter(vine_table['vine'] == 'Y').filter(vine_table['star_rating'] == '5').count()
total_paid_reviews = vine_table.filter(vine_table['vine'] == 'Y').count()

total_fivestar_paid_reviews / total_paid_reviews

0.433508142423406

In [62]:
# Percentage of unpaid (vine = N) 5 star ratings over all ratings
total_fivestar_unpaid_reviews = vine_table.filter(vine_table['vine'] == 'N').filter(vine_table['star_rating'] == '5').count()
total_unpaid_reviews = vine_table.filter(vine_table['vine'] == 'N').count()

total_fivestar_unpaid_reviews / total_unpaid_reviews

0.5953096486616385